<a href="https://colab.research.google.com/github/faizanindyo/FaizAnd/blob/main/Tugas%201/Kaggle/Exercise_Your_First_Machine_Learning_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'home-data-for-ml-course:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F10211%2F111096%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240306%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240306T141719Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dade7cb4d0d8e583b227a956cd21934fccb549e4c36f033510cc8f1f9eedc23a0c0e539113cf3ae5fdc425989b22eac8c2d6226fa4bfe85ba8b14f2edf1d55083fd1edcf4c6326427a12c5e6750e5e86579c34e078e20e60c9b664c365b01507bf777bc432f56a677636bd73f7df9da3362765f6992393596dbe1ec239340493041baff4ffb344c0ff1cbe931553174b95cb37bc3e2f9ca9e0876b6af7a9345fa5998746cf510e0ad87cb5a87ebfdd5d5c80cfdfb750140513dd52889f7c0cd917e9c0f7391cebce498349bd98ada2af471e0bd40db2e358b8fde924bf6105d6219f3d3bf57176f0423fa0dd69efbdd6b080737c09d93935489fff4e15c35d76e,mobile-price-classification:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F11167%2F15520%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240306%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240306T141719Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D01578c5ae02ddd4c66ef3fe2591ac684eb08531b0ecfcca2195a7fe0fc9ae42344dac3dcbce2ed9283e1302511dcf3ccbc2da9e896589434c950158fe4e1c3e96b7f1829051c666a7aae7e9361fd54e3de734d5aaefc02ed07ec8340be1d7684e9787da6aebc014c436b04da870c7fe00985557c9b202525c8e1e070a362f12a98849a989408b0edb45aade85bd15fec41f0378c08c9e140ad128482a5f1491d4850fa5298a24a5439d445c1d772c3627b364de358a8b9cc9f712d6b50491ef7d5fd85005eb24b0cf51f3dea8db24ab7a1b5f0cf13f22dd7c56ec9c5a6063b157086f11069d6005aed976efcd81b162b879de8a1fa187de2fa811401dd6625c1,melbourne-housing-snapshot:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2709%2F38454%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240306%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240306T141719Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D9f4444c9d4818cfc34c47cedc96df7bddafe5e45418c742b769292840fa5953187c82f2a793e69fcefbf650901ddef01ed4140c60bd9006653be64ad62c4347ae19a53c289a37b175cc5e64c00fb48659e5f09aaa33108eac8bcd27c2fc6a50da98c15b3d73b20e9b09b503b3f632e87ef0fe78dfcb8cf2844013768c88424531d25c2b988992069dde7b91a6d3d7e900513661e7554bbe4559199cfa8af32887d24d17590c1cd0fbb0d04c4dc89baf59a368ef789d74948444d25ea882c36de37feacba6cc1e560a88d87d0279e5d292d95037cf288377362a4e14d40a19957ec991aaa31cde218aff97a585ee8a1ae3f3eda38ef5a60087562128659e2ab8d'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Introduction to Machine Learning](https://www.kaggle.com/learn/intro-to-machine-learning) course.  You can reference the tutorial at [this link](https://www.kaggle.com/dansbecker/your-first-machine-learning-model).**

---


## Recap
So far, you have loaded your data and reviewed it with the following code. Run this cell to set up your coding environment where the previous step left off.

In [ ]:
# Code you have previously used to load data
import pandas as pd

# Path of the file to read
iowa_file_path = '../input/home-data-for-ml-course/train.csv'

home_data = pd.read_csv(iowa_file_path)

# Set up code checking
from learntools.core import binder
binder.bind(globals())
from learntools.machine_learning.ex3 import *

print("Setup Complete")

# Exercises

## Step 1: Specify Prediction Target
Select the target variable, which corresponds to the sales price. Save this to a new variable called `y`. You'll need to print a list of the columns to find the name of the column you need.


In [ ]:
# print the list of columns in the dataset to find the name of the prediction target
home_data.columns

In [ ]:
y = home_data.SalePrice

# Check your answer
step_1.check()

In [ ]:
# The lines below will show you a hint or the solution.
# step_1.hint()
# step_1.solution()

## Step 2: Create X
Now you will create a DataFrame called `X` holding the predictive features.

Since you want only some columns from the original data, you'll first create a list with the names of the columns you want in `X`.

You'll use just the following columns in the list (you can copy and paste the whole list to save some typing, though you'll still need to add quotes):
  * LotArea
  * YearBuilt
  * 1stFlrSF
  * 2ndFlrSF
  * FullBath
  * BedroomAbvGr
  * TotRmsAbvGrd

After you've created that list of features, use it to create the DataFrame that you'll use to fit the model.

In [ ]:
# Create the list of features below
feature_names = [
    "LotArea",
    "YearBuilt",
    "1stFlrSF",
    "2ndFlrSF",
    "FullBath",
    "BedroomAbvGr",
    "TotRmsAbvGrd",
]

# Select data corresponding to features in feature_names
X = home_data[feature_names]

# Check your answer
step_2.check()

In [ ]:
# step_2.hint()
# step_2.solution()

## Review Data
Before building a model, take a quick look at **X** to verify it looks sensible

In [ ]:
# Review data
# print description or statistics from X
print(X)

# print the top few lines
#print(X.head())

## Step 3: Specify and Fit Model
Create a `DecisionTreeRegressor` and save it iowa_model. Ensure you've done the relevant import from sklearn to run this command.

Then fit the model you just created using the data in `X` and `y` that you saved above.

In [ ]:
from sklearn.tree import DecisionTreeRegressor
#specify the model.
#For model reproducibility, set a numeric value for random_state when specifying the model
iowa_model = DecisionTreeRegressor(random_state = 1)

# Fit the model
iowa_model.fit(X,y)

# Check your answer
step_3.check()

In [ ]:
# step_3.hint()
# step_3.solution()

## Step 4: Make Predictions
Make predictions with the model's `predict` command using `X` as the data. Save the results to a variable called `predictions`.

In [ ]:
predictions = iowa_model.predict(X)
print(predictions)

# Check your answer
step_4.check()

In [ ]:
# step_4.hint()
# step_4.solution()

## Think About Your Results

Use the `head` method to compare the top few predictions to the actual home values (in `y`) for those same homes. Anything surprising?


In [ ]:
# You can write code in this cell
# You can write code in this cell
print(predictions)
print(y.head())

It's natural to ask how accurate the model's predictions will be and how you can improve that. That will be you're next step.

# Keep Going

You are ready for **[Model Validation](https://www.kaggle.com/dansbecker/model-validation).**


---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/intro-to-machine-learning/discussion) to chat with other learners.*